# Introduction to Decorators
- Specify management or augmentation code for functions and classes
    - Rebinding function and class to other callables at the end of **def** and **class**
- Decorators themselves take the form of callable objects
- Do not need new-style class
- A kind of runtime declaration

    
# Function Decorator VS Class Decorator
- Function decorator
    - Only a specific function
- Class decorator
    - All instance creation calls
    
    
# Why Decorators?
## Good Parts

- Explicity syntax
    - Make intent clearer
- Mainability
    - Not necessary to add extra code at every calll to the class or function
- Consistency
    
## Drawbacks
- Type Change
    - The function or class does not retain its original type after decorated
    - It's rebound to a wrapper object
- Extra call (costs)
- All or nothing
    - Apply to every later call to the decorated object

# Basics

## Function Decorators

### Usage

```python
@decorator
def func(arg):
    pass

func(1)
```

#### is equivalent to

```python
def func(arg):
    pass
    
fun = decorator(fun)
 
func(a)
```

### Implementaion
A decorator itself is a callable that return a callable

In [1]:
# Neseted function (Support both function and methods)
# This is just a common pattern, not the only way
def decorator(fun):
    def wrapper(*args):
        print("[Args]: ", args)
        return fun(*args)

    return wrapper


# Function decorator
@decorator
def func(x, y):
    pass


func(6, 7)


class C:
    @decorator
    def method(self, x, y):
        pass


c = C()
c.method(1, 2)

[Args]:  (6, 7)
[Args]:  (<__main__.C object at 0x1043b1a58>, 1, 2)


## Class Decorators
- Used when you have to make more than one instance of a class, and want to apply the augmentation to every instance 
- **`__getattr__`** based tracing wrapper will not trace and propagate operator overloading calls for built-ins in Python3
    - To work the same in Python3 operator overloading methods generally must be redeined redundantly in the wrapper clas either by hand, by tools or by definition in superclass 


### Usage

```python
@decorator
class C:
    pass

c = C(1)
```

#### is equivalent to

```python
class C:
    pass
    
C = decorator(C)
 
c = C(1)
```

### Implementation
Just like function decorators, though some may involve two levels of augmentation

In [2]:
def decorator(cls):
    class Wrapper:
        def __init__(self, *args):
            self.wrapped = cls(*args)
            self.calls = 0

        def __getattr__(self, name):
            self.calls += 1
            print("[name]: ", name)
            print("[calls]: ", self.calls)
            return getattr(self.wrapped, name)

    return Wrapper


@decorator
class C:
    def __init__(self, x, y):
        self.attr = "spam"


c1 = C(6, 7)
print(c1.attr)

c2 = C(7, 8)
print(c2.attr)

[name]:  attr
[calls]:  1
spam
[name]:  attr
[calls]:  1
spam


- Supporting multiple instance
    - Handles multiple decorated classes (each makes a new Decorator instnace)

In [3]:
class Decorator:
    def __init__(self, C):  # On @decoration
        self.C = C
        self.calls = 0

    def __call__(self, *args):  # On instance creation
        self.calls += 1
        print("[Args]: ", args)
        print("[Calls]: ", self.calls)
        self.wrapped = self.C(*args)
        return self

    def __getattr__(self, attrname):
        return getattr(self.wrapped, attrname)


@Decorator
class C:
    def __init__(*args):
        pass


c1 = C(1)
c2 = C(2)

[Args]:  (1,)
[Calls]:  1
[Args]:  (2,)
[Calls]:  2


### Example (singleton)

In [4]:
def singleton(cls):
    instance = None

    def on_call(*args, **kwargs):
        nonlocal instance
        if not instance:
            instance = cls(*args, **kwargs)
        return instance

    return on_call

In [5]:
@singleton
class C:
    def __init__(self, data):
        self.data = data

    def display(self):
        print(self.data)


c1 = C(1)
c1.display()

# Cannot create a new one
c2 = C(2)
c2.display()

1
1


## Decorator Nesting

```python
@A
@B
@C
def fun(arg):
    pass
```

#### is equivalent to

```python
def fun(arg):
    pass
    
fun = A(B(C(fun)))
```

## Decorator Arguments
- Usually used to
    - retain state information for use in later calls
    - attribute initialization
    - atttribute names to be validated
- It often implied three levels of callables

### Usage
```python
@decorator(x, y)
def fun(arg):
    pass
    
fun(1)
```

#### is equivalent to

```python
def fun(arg):
    pass
    
fun = decorator(x, y)(fun)

fun(1)
```

### Implementation

```python
def decorator(x, y):
    # save or use x, y
    def actualDecorator(func):
        # save or use function func
        # return a callable: nested def, class with __call__ etc.
        return callable
    return actualDecorator
    
```

## Decorator VS Manger Functions

### Manger Function (For one instance)


```python
class Spam
    pass
   
food = Wrapper(Spam)()
```

### Decorator (For all instance)

```python
@Tracer
class Spam:
    pass
    
food = Spam()
```

---

# Function introspection
(May be used to implement a decorator)

In [6]:
def func(a, b, c=1, d=2):
    x = 3
    y = 4


code = func.__code__

In [7]:
code.co_nlocals

6

In [8]:
code.co_varnames

('a', 'b', 'c', 'd', 'x', 'y')

In [9]:
code.co_varnames[: code.co_argcount]

('a', 'b', 'c', 'd')